In [ ]:
# Importing libraries
import numpy as np

from tqdm import tqdm

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from IPython.display import display, HTML

from scipy.optimize import curve_fit
from scipy.constants import c, G, astronomical_unit as AU
from scipy.stats import t as t 

from uncertainties import ufloat, unumpy as un, umath as um


In [ ]:
# Plot settings
plt.rc("font", size=10)
plt.rcParams['font.family'] = 'Liberation Serif'
plt.rcParams['mathtext.rm'] = 'Liberation Serif'
plt.rcParams['mathtext.default'] = 'rm'

In [ ]:
# Constants
M_sun = 1.9884e30  # kg

# Mercury parameters
M_mercury = 0.3301e24  # kg
T_mercury = 87.969  # days
semi_major_axis = 57.909e9  # meters
eccentricity = 0.2056

T_rel = 365.256 / T_mercury # Earth years per Mercury year

phi_teor = ufloat(45.03, 0.03) # arcseconds per century

In [ ]:
# Calculating perehelion distans and initial velocity at perihelion using Kepler's equation
perihelion_distance = semi_major_axis * (1 - eccentricity)

print(f"perihelion_distance = {perihelion_distance:.2e} m")

v_perihelion = np.sqrt(G * M_sun * (1 + eccentricity) / (semi_major_axis * (1 - eccentricity)))

print(f"v_perihelion = {v_perihelion:.2e} m/s")

# Calculating base acceleration at perihelion
a_perihelion = G * M_sun / perihelion_distance**2

print(f"a_perihelion = {a_perihelion:.2e} m/s^2")

# Initial conditions
r0 = np.array([perihelion_distance, 0, 0])  # Mercury starts at perihelion
v0 = np.array([0, v_perihelion, 0])  # tangential velocity at perihelion

In [ ]:
# Functions 
def uncert(data_input, uncert_inst):
    t_coeff = t.ppf((1 + 0.6827)/2, len(data_input)-1)
    return np.sqrt((np.std(data_input)/np.sqrt(len(data_input)))**2 + uncert_inst**2)*t_coeff

def time_parameters(N_T, N_dt):
    T = N_T * T_mercury # iteractions in days
    dt = (2 * v_perihelion / a_perihelion) / 86400 / N_dt # days

    steps = int(T / dt)
    # print(f"Iteractionals time = {T/365.26:2f} years, dt = {dt:2f} days, N_steps = {steps}")
    
    return T, dt, steps

def acceleration(r, v, alpha, beta):
    r_mag = np.linalg.norm(r)  # Compute the magnitude of the position vector
    r_unit = r / r_mag  # Unit vector in the direction of r
    
    # Compute angular momentum squared term: (r vec * r vec dot)^2 / c^2
    L_squared = np.linalg.norm(np.cross(r, v))**2 / c**2
    
    # Compute the relativistic correction factors
    correction_term = 1 + (alpha * 2 * G * M_sun) / (r_mag * c**2) + beta * L_squared / r_mag**2
    
    # Compute the acceleration vector
    a = -G * M_sun / r_mag**2 * correction_term * r_unit

    # r_s = ((2 * G * M_sun) / c**2) / r_mag
    # r_L = L_squared / r_mag**2

    # print(f"r_s = {r_s:.2e}, r_L = {r_L:.2e}, L^2 = {L_squared:.2e}")
    
    return a 

def simulate_orbit(alpha, beta, N_T, N_dt):
    T, dt, steps = time_parameters(N_T, N_dt)
    
    r = np.zeros((steps, 3))
    v = np.zeros((steps, 3))
    
    r[0] = r0
    v[0] = v0
    
    for i in range(steps - 1):
        a = acceleration(r[i], v[i], alpha, beta)

        v[i+1] = v[i] + a * dt * 86400  # days to seconds
        r[i+1] = r[i] + v[i+1] * dt * 86400

    return r

def find_perihelion_positions(r):
    r_magnitude = np.linalg.norm(r, axis=1)
    perihelion_positions = [r[0]]  # Start with the initial perihelion
    uncertainties = []
    
    for i in range(1, len(r_magnitude) - 1):
        if r_magnitude[i] < r_magnitude[i - 1] and r_magnitude[i] < r_magnitude[i + 1]:
            perihelion_positions.append(r[i])
            uncertainty = abs(r_magnitude[i+1] - r_magnitude[i-1]) / 2  # Approximate positional uncertainty
            uncertainties.append(uncertainty)

    first_uncertainty = np.mean(uncertainties)
    uncertainties.insert(0, first_uncertainty)
    
    return np.array(perihelion_positions), np.array(uncertainties)

def compute_perihelion_angles(positions , uncertainties):
    angles_val = []
    angles_err = []
    for i in range(1, len(positions)):
        position_x_1 = ufloat(positions[i-1][0], uncertainties[i-1])
        position_y_1 = ufloat(positions[i-1][1], uncertainties[i-1])

        position_x_2 = ufloat(positions[i][0], uncertainties[i])
        position_y_2 = ufloat(positions[i][1], uncertainties[i])

        phi = um.acos((position_x_1 * position_x_2 + position_y_1 * position_y_2) / (um.sqrt(position_x_1**2 + position_y_1**2) * um.sqrt(position_x_2**2 + position_y_2**2)))

        # angles.append(um.degrees(phi))
        angles_val.append(um.degrees(phi).nominal_value)
        angles_err.append(um.degrees(phi).std_dev)

    average_precession = ufloat(np.mean(angles_val), uncert(angles_val, np.mean(angles_err)))

    return average_precession

def linear_func(x, m, c):
    return m * x + c

def getting_angle_data(coeff, range_min, range_max, step, N_T, N_dt, plot=True):
    if coeff == 'alpha':
        alpha = 0
        beta = range(range_min, range_max, step)
        n_iteractions = len(beta)
    else:
        alpha = range(range_min, range_max, step)
        beta = 0
        n_iteractions = len(alpha)

    # Calculating angles
    angles = []
    angles_arcsec = []
    for i in tqdm(range(n_iteractions), desc="Calculating angles"):
        if coeff == 'alpha':
            r = simulate_orbit(alpha, beta[i], N_T, N_dt)
        else:
            r = simulate_orbit(alpha[i], beta, N_T, N_dt)

        perihelion_positions, perihelion_uncertainties = find_perihelion_positions(r)

        average_precession = compute_perihelion_angles(perihelion_positions, perihelion_uncertainties)

        angles.append(average_precession)

        if coeff == 'alpha':
            angles_arcsec.append(average_precession / beta[i] * 3 * 3600 * T_rel * 100)
        else:
            angles_arcsec.append(average_precession / alpha[i] * 3 * 3600 * T_rel * 100)

    # Fitting the data
    if coeff == 'alpha':
        x_data = beta
    else:
        x_data = alpha

    popt, pcov = curve_fit(linear_func, x_data, un.nominal_values(angles), sigma=un.std_devs(angles), absolute_sigma=True)

    slope, intercept = popt
    slope_err, intercept_err = np.sqrt(np.diag(pcov))

    slope_comp = ufloat(slope, slope_err)

    # Plotting the linear regression

    if plot:
        fig, ax = plt.subplots(figsize=(8, 8))

        if coeff == 'alpha':
            ax.set_xlabel(r'$\beta$')
        else:
            ax.set_xlabel(r'$\alpha$')

        ax.yaxis.label.set_fontsize(20)
        ax.set_ylabel(r'$\varphi ~\text{[deg]}$')
        ax.xaxis.label.set_fontsize(20)
        ax.tick_params(axis="both", which="major", length=10, width=0.5, labelsize=15)
        ax.tick_params(axis="both", which="minor", length=5, width=0.5, labelsize=15)
        ax.xaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
        ax.ticklabel_format(axis='x', style='sci', scilimits=(0,0))

        if coeff == 'alpha':
            ax.errorbar(beta, un.nominal_values(angles), yerr=un.std_devs(angles), fmt='o', markersize=5, capsize=5, color='blue', label='Angles')

            ax.plot(beta, linear_func(beta, slope, intercept), color='red', label=r'$\varphi(\beta) = $' + f'{slope_comp:.2e} ' + r'$\beta$ ' + f'[deg]')
        else:
            ax.errorbar(alpha, un.nominal_values(angles), yerr=un.std_devs(angles), fmt='o', markersize=5, capsize=5, color='blue', label='Angles')

            ax.plot(alpha, linear_func(alpha, slope, intercept), color='red', label=r'$\varphi(\alpha) = $' + f'{slope_comp:.2e} ' + r'$\alpha$ ' + f'[deg]')
            
        ax.legend(fontsize=15)

        plt.tight_layout()  
        if coeff == 'alpha':
            plt.savefig('images/phi(beta).png', bbox_inches='tight')
        else:
            plt.savefig('images/phi(alpha).png', bbox_inches='tight')
        plt.show()

    if coeff == 'alpha':
        return slope_comp, angles_arcsec, beta
    else:
        return slope_comp, angles_arcsec, alpha
    
def plotting_difference(coeff, angles, coeff_range, time):
    fig, ax = plt.subplots(figsize=(10, 6))

    if coeff == 'alpha':
        ax.set_xlabel(r'$\alpha$')
    else:
        ax.set_xlabel(r'$\beta$')
    ax.yaxis.label.set_fontsize(20)
    ax.set_ylabel(r'$\varphi - \varphi_{\text{teor}} ~\text{[arcsec]}$')
    ax.xaxis.label.set_fontsize(20)
    ax.tick_params(axis="both", which="major", length=10, width=0.5, labelsize=15)
    ax.tick_params(axis="both", which="minor", length=5, width=0.5, labelsize=15)
    ax.set_xscale('log')

    for i in range(len(angles)):
        deviation = [angle - phi_teor for angle in angles[i]]

        ax.fill_between(coeff_range[i], un.nominal_values(deviation) - un.std_devs(deviation), un.nominal_values(deviation) + un.std_devs(deviation), color='blue', alpha=0.5, label=f'dt/{time[i]}')

    ax.legend(fontsize=15)

    plt.tight_layout()  
    plt.savefig(f'images/phi({coeff})_diff_{time}.png', bbox_inches='tight')
    plt.show()

In [ ]:
# Runing simulation 
N_T = 20
N_dt = 100

alpha = 0
beta = 10**5

r = simulate_orbit(alpha, beta, N_T, N_dt)

In [ ]:
# Calculating perihelion positions
perihelion_positions, perihelion_uncertainties = find_perihelion_positions(r)

In [ ]:
# Calculating angles between successive perihelions
average_precession = compute_perihelion_angles(perihelion_positions, perihelion_uncertainties)

average_precession_arcsec = average_precession / beta * 3 * 3600 * T_rel * 100

print(f"Average perihelion precession per century: {average_precession_arcsec} arcsec")

In [ ]:
# Getting data for the plot diff phi vs beta
phi_alpha_10, angles_alpha_10, beta_range_10 = getting_angle_data('alpha', 10**4, 10**6, 10**5, N_T, 10, plot=False)
phi_alpha_50, angles_alpha_50, beta_range_50 = getting_angle_data('alpha', 10**4, 10**6, 10**4, N_T, 50, plot=False)
phi_alpha_100, angles_alpha_100, beta_range_100 = getting_angle_data('alpha', 10**4, 10**6, 10**3, N_T, 100, plot=False)
phi_alpha_200, angles_alpha_200, beta_range_200 = getting_angle_data('alpha', 10**4, 10**6, 10**2, N_T, 200, plot=False)
phi_alpha_500, angles_alpha_500, beta_range_500 = getting_angle_data('alpha', 10**4, 10**6, 10, N_T, 500, plot=False)

In [ ]:
plotting_difference('alpha', [angles_alpha_10, angles_alpha_50, angles_alpha_100, angles_alpha_200, angles_alpha_500], [beta_range_10, beta_range_50, beta_range_100, beta_range_200, beta_range_500], [10, 50, 100, 200, 500])

In [ ]:
# Getting data for the plot phi(alpha)
# phi_beta, angles_beta, alpha_range = getting_angle_data('beta', 10**4, 10**6, 10**5, N_T, N_dt)

In [ ]:
# Static plot
fig, ax = plt.subplots(figsize=(10, 10))
ax.set_xlabel('X (AU)')
ax.yaxis.label.set_fontsize(20)
ax.set_ylabel('Y (AU)')
ax.xaxis.label.set_fontsize(20)
ax.tick_params(axis="both", which="major", length=10, width=0.5, labelsize=15)
ax.tick_params(axis="both", which="minor", length=5, width=0.5, labelsize=15)
 
ax.scatter(0, 0, c='yellow', marker='o', s=200, label='Sun')

ax.scatter(perihelion_positions[:, 0] / AU, perihelion_positions[:, 1] / AU, c='b', marker='o', s=20, label='Perihelion')

for pos in perihelion_positions:
    ax.plot([0, pos[0] / AU], [0, pos[1] / AU], 'b--', linewidth=0.5)

ax.plot(r[:, 0] / AU, r[:, 1] / AU, 'r', linewidth=0.2, label='Mercury orbit')

ax.legend(fontsize=15)

plt.tight_layout() 
plt.savefig(f'images/orbit_simulations/orbit_alpha_{alpha}_beta_{beta}_N_T_{N_T}_N_dt_{N_dt}.png', bbox_inches='tight')
plt.show()